In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 850.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 7.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from numpy.linalg import inv
import numpy as np
import math
import cv2
import torch
from matplotlib import pyplot as plt

import torch
from ultralytics import YOLO



In [3]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
model = YOLO('/content/drive/MyDrive/best.pt')  # Ensure you have the correct model file

# Ensure to use the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_

In [5]:
def draw_prediction(img: np.ndarray,
                    class_name: str,
                    df: pd.core.series.Series,
                    color: tuple = (255, 0, 0)):
    '''
    Function to draw prediction around the bounding box identified by the YOLO
    The Function also displays the confidence score top of the bounding box
    '''

    cv2.rectangle(img, (int(df.xmin), int(df.ymin)),
                  (int(df.xmax), int(df.ymax)), color, 2)
    cv2.putText(img, class_name + " " + str(round(df.confidence, 2)),
                (int(df.xmin) - 10, int(df.ymin) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img

In [6]:
class KalmanFilter():
    def __init__(self,
                 xinit: int = 0,
                 yinit: int = 0,
                 fps: int = 30,
                 std_a: float = 0.001,
                 std_x: float = 0.0045,
                 std_y: float = 0.01,
                 cov: float = 100000) -> None:

        # State Matrix
        self.S = np.array([xinit, 0, 0, yinit, 0, 0])
        self.dt = 1 / fps

        # State Transition Model
        # Here, we assume that the model follow Newtonian Kinematics
        self.F = np.array([[1, self.dt, 0.5 * (self.dt * self.dt), 0, 0, 0],
                           [0, 1, self.dt, 0, 0, 0], [0, 0, 1, 0, 0, 0],
                           [0, 0, 0, 1, self.dt, 0.5 * self.dt * self.dt],
                           [0, 0, 0, 0, 1, self.dt], [0, 0, 0, 0, 0, 1]])

        self.std_a = std_a

        # Process Noise
        self.Q = np.array([
            [
                0.25 * self.dt * self.dt * self.dt * self.dt, 0.5 * self.dt *
                self.dt * self.dt, 0.5 * self.dt * self.dt, 0, 0, 0
            ],
            [
                0.5 * self.dt * self.dt * self.dt, self.dt * self.dt, self.dt,
                0, 0, 0
            ], [0.5 * self.dt * self.dt, self.dt, 1, 0, 0, 0],
            [
                0, 0, 0, 0.25 * self.dt * self.dt * self.dt * self.dt,
                0.5 * self.dt * self.dt * self.dt, 0.5 * self.dt * self.dt
            ],
            [
                0, 0, 0, 0.5 * self.dt * self.dt * self.dt, self.dt * self.dt,
                self.dt
            ], [0, 0, 0, 0.5 * self.dt * self.dt, self.dt, 1]
        ]) * self.std_a * self.std_a

        self.std_x = std_x
        self.std_y = std_y

        # Measurement Noise
        self.R = np.array([[self.std_x * self.std_x, 0],
                           [0, self.std_y * self.std_y]])

        self.cov = cov

        # Estimate Uncertainity
        self.P = np.array([[self.cov, 0, 0, 0, 0, 0],
                           [0, self.cov, 0, 0, 0, 0],
                           [0, 0, self.cov, 0, 0, 0],
                           [0, 0, 0, self.cov, 0, 0],
                           [0, 0, 0, 0, self.cov, 0],
                           [0, 0, 0, 0, 0, self.cov]])

        # Observation Matrix
        # Here, we are observing X & Y (0th index and 3rd Index)
        self.H = np.array([[1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]])

        self.I = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0],
                           [0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0],
                           [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])

        # Predicting the next state and estimate uncertainity
        self.S_pred = None
        self.P_pred = None

        # Kalman Gain
        self.K = None

        # Storing all the State, Kalman Gain and Estimate Uncertainity
        self.S_hist = [self.S]
        self.K_hist = []
        self.P_hist = [self.P]

    def pred_new_state(self):
        self.S_pred = self.F.dot(self.S)

    def pred_next_uncertainity(self):
        self.P_pred = self.F.dot(self.P).dot(self.F.T) + self.Q

    def get_Kalman_gain(self):
        self.K = self.P_pred.dot(self.H.T).dot(
            inv(self.H.dot(self.P_pred).dot(self.H.T) + self.R))
        self.K_hist.append(self.K)

    def state_correction(self, z):
        if z == [None, None]:
            self.S = self.S_pred
        else:
            self.S = self.S_pred + +self.K.dot(z - self.H.dot(self.S_pred))

        self.S_hist.append(self.S)

    def uncertainity_correction(self, z):
        if z != [None, None]:
            self.l1 = self.I - self.K.dot(self.H)
            self.P = self.l1.dot(self.P_pred).dot(self.l1.T) + self.K.dot(
                self.R).dot(self.K.T)
        self.P_hist.append(self.P)

def cost_fun(a, b):
    '''
    Cost function for filter Assignment
    Uses euclidean distance for choosing the filter
    '''

    sm = 0
    for i in range(len(a)):
        sm += (a[i] - b[i])**2
    return sm

In [7]:
import json
import numpy as np
import cv2
from collections import Counter

def identify_stationary_objects(threshold=10):
    """
    Identifies stationary objects by grouping similar coordinates based on frequency of occurrence
    and merging those that are close together.
    """
    global global_coord_frequency, stationary_coords

    # Get the list of coordinates and their frequencies from the global frequency dictionary
    coords_with_freq = list(global_coord_frequency.items())

    def group_coordinates_by_proximity(coords_with_freq, threshold):
        """
        Groups coordinates in the global frequency dictionary by proximity and sums their frequencies.
        """
        grouped_freq = []
        used = [False] * len(coords_with_freq)

        for i in range(len(coords_with_freq)):
            if used[i]:
                continue

            current_group = [coords_with_freq[i][0]]  # Start a new group with the current coordinate
            total_freq = coords_with_freq[i][1]  # Initialize total frequency with current frequency
            used[i] = True

            for j in range(i + 1, len(coords_with_freq)):
                # Compute the Euclidean distance between the coordinates
                dist = np.linalg.norm(np.array(coords_with_freq[i][0]) - np.array(coords_with_freq[j][0]))
                if dist < threshold:
                    current_group.append(coords_with_freq[j][0])
                    total_freq += coords_with_freq[j][1]
                    used[j] = True

            # Average the grouped coordinates
            avg_coord = tuple(np.mean(current_group, axis=0))
            grouped_freq.append((avg_coord, total_freq))

        return grouped_freq

    # Group the coordinates by proximity
    grouped_coords_with_freq = group_coordinates_by_proximity(coords_with_freq, threshold)

    # Define a threshold for considering an object stationary based on frequency
    stationary_threshold = 25  # You can adjust this value

    # Find coordinates that occur above the threshold and add to stationary_coords
    stationary_coords = [(coord,freq) for coord, freq in grouped_coords_with_freq if freq > stationary_threshold]

    return stationary_coords

def group_similar_coordinates(coords, threshold=10):
    """
    Groups coordinates that are within a threshold distance from each other.
    Returns a list of unique coordinates (averaged within the group).
    Also updates the global frequency dictionary.
    """
    grouped_coords = []
    used = [False] * len(coords)

    for i in range(len(coords)):
        if used[i]:
            continue

        # Start a new group with the current coordinate
        current_group = [coords[i]]
        used[i] = True

        for j in range(i + 1, len(coords)):
            # Compute the Euclidean distance between coordinates
            dist = np.linalg.norm(np.array(coords[i]) - np.array(coords[j]))
            if dist < threshold:
                current_group.append(coords[j])
                used[j] = True

        # Average the grouped coordinates
        avg_coord = tuple(np.mean(current_group, axis=0))
        grouped_coords.append((avg_coord, len(current_group)))

        # Update global coordinate frequency
        if avg_coord in global_coord_frequency:
            global_coord_frequency[avg_coord] += len(current_group)
        else:
            global_coord_frequency[avg_coord] = len(current_group)

    return grouped_coords


def is_close_to_blacklist(coord, black_list, threshold=1):
    for black_coord in black_list:
        distance = np.sqrt((coord[0] - black_coord[0])**2 + (coord[1] - black_coord[1])**2)
        if distance <= threshold:
            return True
    return False

In [8]:
import cv2
import json
import pandas as pd
from collections import deque, Counter
import numpy as np

number_of_past_frames = 5
def is_shuttle_in_rest(shuttle_coords_queue, number_of_past_frames, threshold=5):

    if len(shuttle_coords_queue) < number_of_past_frames:
        return False  # Not enough data to determine movement

    # Get the minimum and maximum x and y coordinates from the last 20 frames
    x_coords = [coord[0] for coord in shuttle_coords_queue]
    y_coords = [coord[1] for coord in shuttle_coords_queue]

    # Check if the difference between max and min is within the threshold
    if max(x_coords) - min(x_coords) <= threshold and max(y_coords) - min(y_coords) <= threshold:
        return True  # Shuttle is at rest
    else:
        return False  # Shuttle is moving

In [9]:
import cv2
import numpy as np
import json

# def load_tracking_data(json_file_path):
#     """Load tracking data from the specified JSON file."""
#     with open(json_file_path, 'r') as file:
#         data = json.load(file)

#     # Extract court_info and net_info as lists of tuples
#     court_info = [tuple(point) for point in data['court_info']]
#     net_info = [tuple(point) for point in data['net_info']]

#     court_info.sort(key=lambda x: x[1])
#     net_info.sort(key=lambda x: x[1])

#     return court_info, net_info

court_coords=[
        [
            899,
            880
        ],
        [
            2010,
            880
        ],
        [
            799,
            1197
        ],
        [
            2127,
            1197
        ],
        [
            624,
            1724
        ],
        [
            2316,
            1724
        ]
    ]
net_coords=[
        [
            1460,
            810
        ],
        [
            1460,
            1197
        ],
        [
            1490,
            1197
        ],
        [
            1490,
            810
        ]
    ]

def is_shuttle_in_court(shuttle_coord, court_coords, net_coords):
    """Check if the shuttle is inside the court using cv2.pointPolygonTest."""
    # Convert court coordinates to a numpy array of shape (n, 1, 2) required by cv2
    # Court-1 (towards camera)
    court1 = court_coords[4:] + court_coords[2:4]
    court_polygon = np.array(court1, dtype=np.int32).reshape((-1, 1, 2))
    # Check if the point is inside the polygon
    result1 = cv2.pointPolygonTest(court_polygon, shuttle_coord, False)

    # Court-2 (away from camera)
    court2 = court_coords[:2] + court_coords[2:4]
    court_polygon = np.array(court2, dtype=np.int32).reshape((-1, 1, 2))
    # Check if the point is inside the polygon
    result2 = cv2.pointPolygonTest(court_polygon, shuttle_coord, False)

    '''
    1: shuttle in court near camera
    2: shuttle in court away from camera
    False: outside
    '''
    if result1 >= 0:
        return 1  # Shuttle is inside or on the court
    if result2 >= 0:
        return 2
    return False  # Shuttle is outside the court

def determine_shooter(shuttle_coords_deque):
    """Determine which player shot the shuttle using the deque of shuttle coordinates.

    Args:
        shuttle_coords_deque: A deque containing the shuttle coordinates (x, y) for the last 20 frames.

    Returns:
        1: If the shuttle was shot by player 1 (moving downward).
        2: If the shuttle was shot by player 2 (moving upward).
        None: If there's not enough information.
    """
    if len(shuttle_coords_deque) < 2:
        return 0  # Not enough data to determine the shooter

    # Extract y-coordinates from the deque
    y_coords = [coord[1] for coord in shuttle_coords_deque]

    # Calculate the difference between consecutive y-coordinates
    deltas = np.diff(y_coords)  # This will give us the differences between frames

    # Analyze the y-coordinate differences to determine the direction of motion
    avg_delta = np.mean(deltas)  # Get the average direction of movement

    # If avg_delta > 0, shuttle is moving downward (toward player 1), player 2 shot it
    # If avg_delta < 0, shuttle is moving upward (toward player 2), player 1 shot it
    if avg_delta > 0:
        return 2  # Player 2 shot the shuttle (moving toward player 1)
    elif avg_delta < 0:
        return 1  # Player 1 shot the shuttle (moving toward player 2)

    return 0  # No clear direction

def assign_points(shuttle_position, prev_k_frame):
    if shuttle_position == 1:
        score[1] += 1

    elif shuttle_position == 2:
        score[0] += 1

    else:
        shooter = determine_shooter(prev_k_frame)
        if shooter == 2:
            score[0] += 1
        else:
            score[1] += 1

In [29]:
import cv2
import numpy as np
import pandas as pd
import json
from collections import Counter, deque
from scipy.ndimage import uniform_filter1d
# Initialize the global coordinate frequency dictionary
global_coord_frequency = {}
stationary_coords = []
score=[0,0]
relay_flag=0
relay_start_frame = None  # Track the frame where the relay starts

def is_consistently_decreasing(y_coords, window_size=5):
    """
    Check if y-coordinates are consistently decreasing over a window of frames.
    """
    print(y_coords)
    if len(y_coords) < window_size:
        return False
    recent_coords = list(y_coords)[-window_size:]

    return all(recent_coords[i] > recent_coords[i+1] for i in range(window_size-1))

def real_time_detection_and_tracking(video_path, output_json_path):
    global global_coord_frequency, stationary_coords, relay_flag, relay_start_frame

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"FPS: {fps}")

    ret, frame = cap.read()
    frame_height, frame_width = frame.shape[:2]
    out = cv2.VideoWriter('realtime_tracking_kalman.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    tracking_data = {}
    scored = False
    coord_counter = Counter()
    frame_count = 0
    shuttle_coords_queue = deque(maxlen=5)
    prev_k_frame = deque(maxlen=10)
    black_list = [(639.6225493808962, 347.902106806917), (292.45753370012557, 139.51016199021112),
                  (36.80934792376579, 199.977830115785)]
    lastx, lasty, lastframeno = None, None, None

    listt = {}
    speed_history = []
    rest_coords = []
    y_coord_history = deque(maxlen=10)
    # Rest state tracking
    rest_state_counter = 0
    REST_THRESHOLD = 3  # Number of consecutive frames to consider as "at rest"

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        print(f"Processing frame {frame_count}")

        results = model([frame])

        boxes = results[0].boxes.xyxy.cpu().numpy()
        class_ids = results[0].boxes.cls.cpu().int().numpy()
        scores = results[0].boxes.conf.cpu().numpy()

        df_current = pd.DataFrame({
            'xmin': boxes[:, 0],
            'ymin': boxes[:, 1],
            'xmax': boxes[:, 2],
            'ymax': boxes[:, 3],
            'class_id': class_ids,
            'confidence': scores
        })

        current_coords = []
        if frame_count not in listt:
            listt[frame_count] = []

        for _, row in df_current.iterrows():
            coord = [(row['xmin'] + row['xmax']) / 2, (row['ymin'] + row['ymax']) / 2]

            if row['class_id'] == 0:
                if not is_close_to_blacklist(coord, black_list, threshold=15):
                    current_coords.append(coord)

                    speed = 0 if lastx is None else np.sqrt((coord[0] - lastx)**2 + (coord[1] - lasty)**2) / (frame_count - lastframeno)
                    listt[frame_count].append({
                        'x_center': coord[0],
                        'y_center': coord[1],
                        'speed': speed
                    })

                    lastx, lasty, lastframeno = coord[0], coord[1], frame_count

        # Update tracking_data and check for rest state
        is_at_rest = False
        if frame_count in listt and len(listt[frame_count]) == 1:
            coord = (listt[frame_count][0]['x_center'], listt[frame_count][0]['y_center'])
            shuttle_coords_queue.append(coord)
            y_coord_history.append(coord[1])

            # Relay start detection
            if relay_flag == 0 and is_consistently_decreasing(y_coord_history):
                relay_flag = 1
                relay_start_frame = frame_count  # Track when the relay starts
                print(f"Relay start at frame {frame_count}")

            if is_shuttle_in_rest(shuttle_coords_queue, 5):
                rest_state_counter += 1
                if rest_state_counter >= REST_THRESHOLD:
                    is_at_rest = True
                    rest_coords.append(coord)
                if is_at_rest and relay_flag == 1:
                    relay_flag = 0
                    relay_start_frame = None  # Reset relay start frame

            else:
                rest_state_counter = 0

            speed_history.append(listt[frame_count][0]['speed'])
            if len(speed_history) > 5:
                speed_history.pop(0)
            smoothed_speed = np.mean(speed_history)

            tracking_data[f"{frame_count}"] = {
              'x_center': coord[0],
              'y_center': coord[1],
              'smoothened_speed': smoothed_speed,
              'is_at_rest': is_at_rest,
              'relay_active': relay_flag == 1
          }

        grouped_coords = group_similar_coordinates(current_coords, threshold=100)
        rest_coords = group_similar_coordinates(rest_coords, threshold=10)
        if rest_coords:
            rest_coords = [rest_coords[0][0]]

        coord_counter.clear()
        for coord, count in grouped_coords:
            coord_counter[coord] += count

        # Visualization
        tmp_img = frame.copy()
        dummy = 15

        for x, y in current_coords:
            cv2.rectangle(tmp_img, (int(x) - dummy, int(y) - dummy),
                          (int(x) + dummy, int(y) + dummy), (0, 255, 0), 2)

            if str(frame_count) in tracking_data:
                font = cv2.FONT_HERSHEY_SIMPLEX
                speed = tracking_data[str(frame_count)]['smoothened_speed']
                speed_text = f"Speed: {speed:.2f}"
                cv2.putText(tmp_img, speed_text, (int(x) - dummy, int(y) - dummy - 10),
                            font, 0.5, (0, 140, 255), 2)

        # Draw black list rectangles
        for x, y in black_list:
            cv2.rectangle(tmp_img, (int(x) - dummy, int(y) - dummy),
                          (int(x) + dummy, int(y) + dummy), (0, 140, 255), 5)
            cv2.putText(tmp_img, 'stationary', (int(x) - dummy, int(y) - dummy - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 140, 255), 2)

        # Draw rest state indicator
        if is_at_rest:
            if not scored:
              shuttle_position = is_shuttle_in_court(coord, court_coords, net_coords)
              assign_points(shuttle_position, prev_k_frame.copy())
              scored = True
            last_rest_coord = rest_coords[-1]
            text_position = (int(last_rest_coord[0]), int(last_rest_coord[1]) - 30)
            cv2.putText(tmp_img, 'Shuttle is at rest', text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 4)
            cv2.putText(tmp_img, 'Shuttle is at rest', text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Calculate relay time
        if relay_flag == 1 and relay_start_frame is not None:
            relay_duration = frame_count - relay_start_frame
            relay_text = f"Relay Time: {relay_duration} frames"
        else:
            relay_text = "Relay Inactive"

        cv2.putText(tmp_img, relay_text, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Position for score[1] (Player 2's score) at the top-left corner
        top_left_position = (50, 50)  # (x, y) coordinates for top-left corner

        # Position for score[0] (Player 1's score) at the bottom-left corner
        bottom_left_position = (50, frame_height - 50)  # (x, y) coordinates for bottom-left corner

        # Draw Player 2's score (top-left)
        cv2.putText(tmp_img, f"Player 2: {score[1]}", top_left_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 5)

        # Draw Player 1's score (bottom-left)
        cv2.putText(tmp_img, f"Player 1: {score[0]}", bottom_left_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 5)


        out.write(tmp_img)
        prev_k_frame.append(coord)
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the real-time shuttlecock detection and tracking


In [30]:
real_time_detection_and_tracking('/content/drive/MyDrive/drop.mp4', 'tracking_data.json')

FPS: 24.93188276682772
Processing frame 0

0: 384x640 1 Shuttlecock, 1786.4ms
Speed: 20.1ms preprocess, 1786.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 1

0: 384x640 2 Shuttlecocks, 1727.4ms
Speed: 3.4ms preprocess, 1727.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2

0: 384x640 2 Shuttlecocks, 1583.3ms
Speed: 3.3ms preprocess, 1583.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 3

0: 384x640 2 Shuttlecocks, 1159.3ms
Speed: 4.5ms preprocess, 1159.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 4

0: 384x640 2 Shuttlecocks, 957.9ms
Speed: 2.4ms preprocess, 957.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 5

0: 384x640 2 Shuttlecocks, 966.4ms
Speed: 4.8ms preprocess, 966.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 6

0: 384x640 2 Shuttlecocks, 953.

<ipython-input-29-990d267fe972>:87: RuntimeWarning: divide by zero encountered in scalar divide
  speed = 0 if lastx is None else np.sqrt((coord[0] - lastx)**2 + (coord[1] - lasty)**2) / (frame_count - lastframeno)


0: 384x640 1 Shuttlecock, 1226.2ms
Speed: 4.7ms preprocess, 1226.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 64

0: 384x640 1 Shuttlecock, 1492.4ms
Speed: 4.5ms preprocess, 1492.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 65

0: 384x640 1 Shuttlecock, 1487.7ms
Speed: 3.7ms preprocess, 1487.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 66

0: 384x640 1 Shuttlecock, 1458.3ms
Speed: 3.5ms preprocess, 1458.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 67

0: 384x640 1 Shuttlecock, 949.1ms
Speed: 4.4ms preprocess, 949.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 68

0: 384x640 1 Shuttlecock, 955.0ms
Speed: 4.1ms preprocess, 955.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 69

0: 384x640 2 Shuttlecocks, 942.6ms
Speed: 3.9ms preprocess, 942.6ms infere

In [31]:
# prompt: save realtime_tracking_kalman.mp4 in drive using shutils. Remember shutils has no attribute copy

import shutil

# Save the output video to Google Drive
shutil.move('realtime_tracking_kalman.mp4', '/content/drive/My Drive/realtime_tracking_kalman.mp4')


'/content/drive/My Drive/realtime_tracking_kalman.mp4'